In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras as K
print(tf.__version__)


In [ ]:
import p.s as ss
from p.s import S
if __name__ == '__main__':
    #check()
    s=S() # python is brain damaged
    #time,series=s.generateTimeSeriesDataForWeek1()
    time,series=ss.week1() # this works, why doesn't the.
    
    s=S(time,series)
    #print(time,series) # prints start and end.
    if False:
        s.checkWeek1Series()
    train,validate,test=s.split(val=S.SPLIT_TIME)
    print("entire:",len(s.series))
    print("train:",len(train.series))
    print("validate:",len(validate.series))
    print("test:",len(test.series))
    print("entire")
    print(s.series[:5])
    print(s.series[-5:])
    print("train:")
    print(train.series[:5])
    print(train.series[-5:])
    print("validate")
    print(validate.series[:5])
    print(validate.series[-5:])



In [ ]:
    %%capture
    s.plot0(title='Original Series')


In [ ]:
    %%capture
    train.plot0(title='Training Series')
    validate.plot0(title='Validation Series')

In [ ]:
print(validate.time[:5],validate.series[:5])
print(validate.time[-5:],validate.series[-5:])

In [ ]:
def plot_series(time, series, format="-", title="", label=None, start=0, end=None):
    """Plot the series"""
    plt.plot(time[start:end], series[start:end], format, label=label)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.title(title)
    if label:
        plt.legend()
    plt.grid(True)
    
ss.testComputeMetrics()
naive_forecast =  s.series[S.SPLIT_TIME-1:-1]
forecast=S(validate.time,naive_forecast) # is this correct?
print(f"validation series has shape: {validate.series.shape}\n")
print(f"naive forecast has shape: {naive_forecast.shape}\n")
print(f"comparable with validation series: {validate.series.shape == naive_forecast.shape}")



In [ ]:
 %%capture
plt.figure(figsize=(10, 6))
#plot_series(validate.time,validate.series, label="validation set")
validate.plot0(title="validation set")
print("forcast:",forecast.time.shape,forecast.series.shape)
forecast.plot0(title="naive forecast")
#plot_series(forecast.time, forecast.series, label="naive forecast")


In [ ]:
 %%capture
plt.figure(figsize=(10, 6))
plot_series(validate.time,validate.series,start=330, end=361, label="validation set")
plot_series(forecast.time, forecast.series,start=330, end=361, label="naive forecast")


In [ ]:
print("validate")
print(validate.series[:5])
print(validate.series[-5:])
print("forecast")
print(forecast.series[:5])
print(forecast.series[-5:])
sum=(validate.series-forecast.series).sum()
print("sum of errors:",(validate.series-forecast.series).sum())
mse, mae = validate.compute_metrics(forecast)
print(f"mse: {mse:.2f}, mae: {mae:.2f} for naive forecast")
# Expected Output: mse: 19.58, mae: 2.60 for naive forecast
expected=3.796112

if np.abs(sum-expected)>=1e-7:
    print("ex",expected,", ac:",sum)
print("error:",sum-expected)



In [ ]:
# Test your function
window_size=30
moving_avg =ss. moving_average_forecast(s.series, window_size)
print(f"moving average forecast with whole SERIES has shape: {moving_avg.shape}\n")

# Slice it so it matches the validation period
moving_avg = moving_avg[S.SPLIT_TIME - window_size:]
print(f"moving average forecast after slicing has shape: {moving_avg.shape}\n")
print(f"comparable with validation series: {validate.series.shape == moving_avg.shape}")
ma=S(validate.time,moving_avg)
print("ma:",ma.time.shape,ma.series.shape)
plt.figure(figsize=(10, 6))
validate.plot_series(label="validation set")
#F.plot_series(time_valid, series_valid)
ma.plot_series(label="moving average")
#F.plot_series(time_valid, moving_avg)

In [ ]:
# Compute evaluation metrics
#mse, mae = compute_metrics(series_valid, moving_avg)
mse, mae = validate.compute_metrics(ma)

print(f"mse: {mse:.2f}, mae: {mae:.2f} for moving average forecast")
# Expected Output:  mse: 65.79, mae: 4.30 for moving average forecast



In [ ]:
# differences
#diff_series =  (SERIES[365:] - SERIES[:-365])
#print("first vale:",diff_series,type(diff_series))
#diff_time = TIME[365:]
ds=s.series[365:] - s.series[:-365]
dt=s.time[365:]
diff=S(dt,ds)

### END CODE HERE

print(f"Whole SERIES has {len(s.series)} elements so the differencing should have {len(s.series)-365} elements\n")
print(f"diff series has shape: {diff.series.shape}\n")
print(f"x-coordinate of diff series has shape: {diff.time.shape}\n")

plt.figure(figsize=(10, 6))
plot_series(diff.time, diff.series)
plt.show()

In [ ]:
# Apply the moving avg to diff series
magic50=50
diff_moving_avg = ss.moving_average_forecast(diff.series, magic50)
print(f"moving average forecast with diff series has shape: {diff_moving_avg.shape}\n")
# Perform the correct slicing
diff_moving_avg = diff_moving_avg[1100-365-magic50:]
print(f"moving average forecast with diff series after slicing has shape: {diff_moving_avg.shape}\n")
print(f"comparable with validation series: {validate.series.shape == diff_moving_avg.shape}")


plt.figure(figsize=(10, 6))
plot_series(validate.time, diff.series[1100 - 365:])
plot_series(validate.time, diff_moving_avg)
plt.show()

In [ ]:
# Slice the whole SERIES to get the past values
#print(SPLIT_TIME,SPLIT_TIME-365)
#past_series = SERIES[SPLIT_TIME-365:-365]
past_series = s.series[S.SPLIT_TIME-365:-365]
print(f"past series has shape: {past_series.shape}\n")
# Add the past to the moving average of diff series
diff_moving_avg_plus_past = past_series + diff_moving_avg
### END CODE HERE

print(f"moving average forecast with diff series plus past has shape: {diff_moving_avg_plus_past.shape}\n")
print(f"comparable with validation series: {validate.series.shape == diff_moving_avg_plus_past.shape}")

plt.figure(figsize=(10, 6))
plot_series(validate.time, validate.series)
plot_series(validate.time, diff_moving_avg_plus_past)
plt.show()

In [ ]:
# Compute evaluation metrics
diffMAPlusPast=S(validate.time,diff_moving_avg_plus_past)
print(diffMAPlusPast.time.shape,diffMAPlusPast.series.shape)
#mse, mae = compute_metrics(series_valid, diff_moving_avg_plus_past)
mse, mae = validate.compute_metrics(diffMAPlusPast)
print(f"mse: {mse:.2f}, mae: {mae:.2f} for moving average plus past forecast")
# Expected Output: mse: 8.50, mae: 2.33 for moving average plus past forecast



In [ ]:
# Perform the correct split of SERIES
#smooth_past_series = moving_average_forecast(SERIES[SPLIT_TIME - 370:-360], 10)
smooth_past_series = ss.moving_average_forecast(s.series[S.SPLIT_TIME - 370:-360], 10)

print(f"smooth past series has shape: {smooth_past_series.shape}\n")
# Add the smoothed out past values to the moving avg of diff series
diff_moving_avg_plus_smooth_past = smooth_past_series +  diff_moving_avg
print(f"moving average forecast with diff series plus past has shape: {diff_moving_avg_plus_smooth_past.shape}\n")
print(f"comparable with validation series: {validate.series.shape == diff_moving_avg_plus_smooth_past.shape}")

plt.figure(figsize=(10, 6))
plot_series(validate.time, validate.series)
plot_series(validate.time, diff_moving_avg_plus_smooth_past)
plt.show()

In [ ]:
# Compute evaluation metrics
smooth=S(validate.time,diff_moving_avg_plus_smooth_past)
#mse, mae = compute_metrics(series_valid, diff_moving_avg_plus_smooth_past)
mse, mae = validate.compute_metrics(smooth)
print(f"mse: {mse:.2f}, mae: {mae:.2f} for moving average plus smooth past forecast")
# Expected Output: mse: 12.53, mae: 2.20 for moving average plus smooth past forecast



In [ ]:
def create_model(window_size=S.WINDOW_SIZE):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(10, input_shape=[window_size], activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1)
        ])
    optimizer = tf.keras.optimizers.SGD(momentum=0.9)
    model.compile(loss="mse", optimizer= tf.keras.optimizers.SGD(learning_rate=4.0e-6, momentum=0.9))
    return model

dataset = S.windowed_dataset(sT)
model = create_model()
model.fit(dataset, epochs=115,verbose=0)
layers=model.layers
print(len(layers),type(layers[0]))
l0=layers[0]
print("Layer weights {}".format(l0.get_weights()))

In [ ]:
# week 2
if False:
    s=S.create()
    tT,sT,tV,sV,_,_ = s.split(val=S.SPLIT_TIME)
    plt.figure(figsize=(10, 6))
    s.plot()
    plt.show()
